In [164]:
# Устанавливаем дополнительные модули
!pip install pickle5 nltk IPython nltk;

You should consider upgrading via the '/home/ls/PycharmProjects/FlaskML/venv/bin/python -m pip install --upgrade pip' command.


In [178]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import json
import warnings
import dill
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')
%matplotlib inline

### Импорт данных

In [167]:
import pickle5 as pickle
with open ('./data/reviews_tonality_X.pkl', 'rb') as fh:
    X = pickle.load(fh)
with open ('./data/reviews_tonality_y.pkl', 'rb') as fh:
    y = pickle.load(fh)
X['tonality'] = y

In [169]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=147)

In [170]:
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [171]:
class ColumnSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [172]:
features = ['text']
target = 'tonality'

In [173]:
feats = Pipeline([
                ('imputer', TextImputer('text', '')),
                ('selector', ColumnSelector(key='text')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])

In [175]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: user 3.32 s, sys: 40 ms, total: 3.36 s
Wall time: 4.17 s


Pipeline(steps=[('features',
                 Pipeline(steps=[('imputer', TextImputer(key='text', value='')),
                                 ('selector', ColumnSelector(key='text')),
                                 ('tfidf',
                                  TfidfVectorizer(max_df=0.9, min_df=10))])),
                ('classifier', LogisticRegression())])

In [176]:
pipeline.steps

[('features',
  Pipeline(steps=[('imputer', TextImputer(key='text', value='')),
                  ('selector', ColumnSelector(key='text')),
                  ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))])),
 ('classifier', LogisticRegression())]

In [179]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)